# Optimización entera: problemas de ajedrez

Un ejemplo típico de IP es aquel en el que se usan variables enteras (en este caso binarias) para decidir si se pone o no [una determinada pieza de ajedrez en cada casilla](https://en.wikipedia.org/wiki/Mathematical_chess_problem), de acuerdo a distintas restricciones según lo que busquemos.

Antes de nada, instalamos `cplex` y `docplex` si es que no los tenemos ya:

In [ ]:
!pip install cplex docplex

## Problema de las torres

Dado un tablero de ajedrez de tamaño $N \times N$, queremos ver **cuál es el número máximo de torres $T$ que podemos poner en él, de forma que no se ataquen entre sí**.

Para ello, hay que recordar que en el ajedrez una torre sólo se puede mover vertical u horizontalmente. Por ejemplo, si tomamos el valor habitual de $N = 8$ (tablero $8 \times 8$), y empezamos a numerar por la esquina superior izquierda, vemos que el colocar una torre en la fila 3, columna 5, implica que el resto de torres que pongamos no podrán estar en ninguna de las casillas de esa fila ni de esa columna (puesto que las atacaría esta torre que acabamos de poner):

<center>
<img width=250, src="https://acm.cs.nthu.edu.tw/media/uploads/2021/02/26/p9vjszg.png">
</center>

Vamos a construir una función `rooks_problem(n)` que, a partir del valor de $N$, nos devuelva el modelo `cplex` correspondiente:


In [ ]:
from docplex.mp.model import Model

def rooks_problem(n: int) -> Model:
  """Construye el problema de maximizar el nº de torres en un tablero NxN de forma que no se ataquen entre sí"""
  prob = Model(name=f"Rooks_problem_{n}x{n}")
  rows = cols = range(1, n+1)
  rooks = prob.binary_var_matrix(rows, cols, name="rooks")     # 0 = no hay torre en casilla [fila, columna], 1 = sí que la hay
  prob.add_constraints([(prob.sum(rooks[r, c] for c in cols) <= 1) for r in rows])     # como mucho una torre por fila, repitiendo para todas las filas
  prob.add_constraints([(prob.sum(rooks[r, c] for r in rows) <= 1) for c in cols])     # ídem con las columnas
  n_rooks = prob.sum(rooks[r, c] for r in rows for c in cols)     # total de torres en el tablero (se suman sólo los 1s)
  prob.maximize(n_rooks)    # y buscamos maximizar ese nº
  return prob

Por ejemplo, si probamos con el tamaño habitual $N=8$, vemos que obtenemos un MILP (las variables son enteras binarias, pero las restricciones y el objetivo son lineales) con las 16 restricciones esperadas:

In [ ]:
p = rooks_problem(n=8)
p.print_information()

Y si probamos a resolverlo resulta:

In [ ]:
from docplex.mp.solution import SolveSolution

s: SolveSolution = p.solve()
print(s)

Lo cual nos indica que el nº máximo de torres es 8, y que por ejemplo pueden estar dispuestas siguiendo la línea que va en la dirección Noroeste-Sudeste. Podemos codificar una función que pinte el tablero a partir de la solución obtenida:

In [ ]:
def print_solution(n: int, sol: SolveSolution) -> None:
  """Imprime el tablero NxN con la solución obtenida"""
  if sol is not None:
    print(f"Nº de piezas: {int(sol.objective_value)}")
    line = ''
    for i, v in enumerate(sol.model.iter_variables(), start=1):
      print('|' + ('X' if sol[v] == 1 else ' '), end="")    # X si hay pieza, espacio en blanco si no
      if i % n == 0:   # fin de línea
        print("|")
  else:
    print("El problema no tiene solución")

Así que si la llamamos con la solución obtenida en `s` nos pinta el tablero:

In [ ]:
print_solution(n=8, sol=s)

Hay que acordarse de terminar el modelo, una vez que ya no lo necesitamos:

In [ ]:
p.end()

Podemos también probar qué pasa para distintos valores de $N$ y cómo cambia $T$. El patrón es muy obvio:

In [ ]:
for n in range(1, 11):
  with rooks_problem(n) as prob:    # al usarlo en bloque with ya se llama a end() automáticamente
    print(f"\nSolución para tablero {n}x{n}")
    sol = prob.solve()
    print_solution(n=n, sol=sol)

### Ejercicios adicionales

1. ¿Qué pasaría si en lugar de maximizar $T$ lo minimizaras? Compruébalo en el código.
2. ¿Y si vuelves a maximizar $T$, pero añades la restricción adicional $T \gt N$?

## Problema de los alfiles

Un problema relacionado con el anterior es el de ver **cuántos alfiles $A$ podemos colocar como mucho en un tablero $N \times N$, de forma que no se ataquen entre sí**. Lo que cambia es que los alfiles se mueven diagonalmente. Por ejemplo, si emplazamos un alfil en la fila 4, columna 5, las casillas que se anulan para poder colocar más alfiles son las correspondientes a las 2 diagonales en las que se encuentra este alfil, tanto en la dirección Noroeste-Sudeste (diagonal) como en la dirección Noreste-Sudoeste (antidiagonal):

<center>
<img width=250, src="https://media.geeksforgeeks.org/wp-content/cdn-uploads/chess1.png">
</center>

Ahora la función `bishops_problem` es bastante parecida a `rooks_problem`, sólo que debemos prestar especial atención a qué casillas tenemos en cuenta diagonalmente para las restricciones:

In [ ]:
def bishops_problem(n: int) -> Model:
  """Construye el problema de maximizar el nº de alfiles en un tablero NxN de forma que no se ataquen entre sí"""
  prob = Model(name=f"Bishops_problem_{n}x{n}")
  rows = cols = range(1, n+1)
  bishops = prob.binary_var_matrix(rows, cols, name="rooks")     # 0 = no hay alfil en casilla [fila, columna], 1 = sí que lo hay
  prob.add_constraints([
      (prob.sum(bishops[r, c] for r in rows for c in cols if r-c == k) <= 1)     # como mucho un alfil por diagonal
      for k in range(-n+1, n)   # cada diagonal cumple que r-c va entre -n+1 (esquina NE) y n-1 (esquina SO), y cuando r-c == 0 es diagonal principal
  ])
  prob.add_constraints([
      (prob.sum(bishops[r, c] for r in rows for c in cols if r+c == k) <= 1)     # como mucho un alfil por antidiagonal
      for k in range(2, 2*n+1)   # cada antidiagonal cumple que r+c va entre 2 (esquina NO) y 2n (esquina SE), y cuando r+c == n+1 es antidiagonal principal
  ])
  n_bishops = prob.sum(bishops[r, c] for r in rows for c in cols)     # total de alfiles en el tablero (se suman sólo los 1s)
  prob.maximize(n_bishops)    # y buscamos maximizar ese nº
  return prob

Ahora el patrón es más interesante que el de antes, y las soluciones obtenidas no son tan triviales:

In [ ]:
for n in range(1, 11):
  with bishops_problem(n) as prob:
    print(f"\nSolución para tablero {n}x{n}")
    sol = prob.solve()
    print_solution(n=n, sol=sol)

### Ejercicios adicionales

1. ¿Cuál es la fórmula de $A$ para un cierto $N$?
2. Puedes ver que en las soluciones anteriores los alfiles siempre se disponen en los bordes del tablero. ¿Qué pasa si introduces la restricción adicional de que al menos un alfil tiene que estar en una casilla del interior del tablero? ¿Cómo varía $A$ en este caso?

## Problema de las reinas

Finalmente, llegamos al problema más conocido de este tipo, que es el de ver **cuántas reinas $R$ podemos llegar a colocar como mucho en un tablero $N \times N$, sin que se puedan comer unas a otras**.

La reina es la pieza más versátil del ajedrez, ya que se puede mover horizontal, vertical o diagonalmente, de forma que en cuanto colocamos una reina, se anulan todas las posibilidades de su fila, columna, diagonal y antidiagonal:

<center>
<img width=250, src="https://www.researchgate.net/profile/Syed-Shah-120/publication/265034297/figure/fig1/AS:525820109299712@1502376691132/Queen-Attacking-Search-Space-The-conventional-N-Queen-encompasses-the-assignments-of_Q320.jpg">
</center>

No tenemos por tanto más que juntar los 2 problemas anteriores, porque una reina al fin y al cabo es como si fuera una torre y un alfil simultáneos:


In [ ]:
def queens_problem(n: int) -> Model:
  """Construye el problema de maximizar el nº de reinas en un tablero NxN de forma que no se ataquen entre sí"""
  prob = Model(name=f"Queens_problem_{n}x{n}")
  rows = cols = range(1, n+1)
  queens = prob.binary_var_matrix(rows, cols, name="queens")     # 0 = no hay reina en casilla [fila, columna], 1 = sí que la hay
  prob.add_constraints([(prob.sum(queens[r, c] for c in cols) <= 1) for r in rows])     # como mucho una reina por fila
  prob.add_constraints([(prob.sum(queens[r, c] for r in rows) <= 1) for c in cols])     # como mucho una reina por columna
  prob.add_constraints([
      (prob.sum(queens[r, c] for r in rows for c in cols if r-c == k) <= 1)     # como mucho una reina por diagonal
      for k in range(-n+1, n)   # cada diagonal cumple que r-c va entre -n+1 (esquina NE) y n-1 (esquina SO), y cuando r-c == 0 es diagonal principal
  ])
  prob.add_constraints([
      (prob.sum(queens[r, c] for r in rows for c in cols if r+c == k) <= 1)     # como mucho una reina por antidiagonal
      for k in range(2, 2*n+1)   # cada antidiagonal cumple que r+c va entre 2 (esquina NO) y 2n (esquina SE), y cuando r+c == n+1 es antidiagonal principal
  ])
  n_queens = prob.sum(queens[r, c] for r in rows for c in cols)     # total de reinas en el tablero (se suman sólo los 1s)
  prob.maximize(n_queens)    # y buscamos maximizar ese nº
  return prob

Y entonces obtenemos:

In [ ]:
for n in range(1, 11):
  with queens_problem(n) as prob:
    print(f"\nSolución para tablero {n}x{n}")
    sol = prob.solve()
    print_solution(n=n, sol=sol)

### Ejercicios adicionales

1. A tenor de los resultados, hay un par de valores de $N$ que rompen el patrón general de cómo evoluciona $R$ con $N$. ¿Cuáles son?

2. Comprueba una curiosa propiedad que se cumple cuando $R = N$. Si numeramos cada casilla con los números del 1 al $N^2$ de izquierda a derecha y de arriba abajo, y sumamos las posiciones de las $R$ casillas en que hay una reina, resulta que esa suma da siempre $S = \frac{N^3 + N}{2}$. Por ejemplo, para $N = 4$ tenemos por simetría estas 2 posibles soluciones:

$$S = 2 + 8 + 9 + 15 = 34$$
$$S = 3 + 5 + 12 + 14 = 34$$
$$S = \frac{4^3 + 4}{2} = \frac{64 + 4}{2} = 34$$

<center><img width=500, src="https://upload.wikimedia.org/wikipedia/commons/a/a2/Tablero_4x4_AAS.png"></img></center>